In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df =pd.read_csv("Air_Quality_dataset_of_kathmandu.csv")

In [ ]:
df

,Date,Time,PM10 (μg/m³),PM2.5 (μg/m³),CO (μg/m³),NO2 (μg/m³),SO2 (μg/m³),Temp (°C),Humidity (%),Wind_Speed (km/h),Soil_Moisture (m³/m³),AQI
0,2022-08-05,00:00:00,31.7,22.2,478.0,14.1,5.2,20.3,98.0,3.1,0.426,58
1,2022-08-05,01:00:00,29.7,20.8,435.0,11.6,4.5,21.0,96.0,1.8,0.425,59
2,2022-08-05,02:00:00,25.3,17.7,374.0,8.2,3.5,21.7,93.0,3.1,0.424,59
3,2022-08-05,03:00:00,23.1,16.2,303.0,4.3,2.5,23.2,86.0,3.1,0.423,60
4,2022-08-05,04:00:00,20.3,14.2,272.0,2.9,2.0,25.1,76.0,2.1,0.421,60
...,...,...,...,...,...,...,...,...,...,...,...,...
22531,2025-02-28,19:00:00,49.1,29.6,780.0,13.4,7.8,NaN,NaN,NaN,NaN,153
22532,2025-02-28,20:00:00,46.4,28.2,652.0,11.1,7.5,NaN,NaN,NaN,NaN,151
22533,2025-02-28,21:00:00,44.2,27.1,548.0,9.6,7.2,NaN,NaN,NaN,NaN,146
22534,2025-02-28,22:00:00,42.4,26.0,489.0,9.1,6.9,NaN,NaN,NaN,NaN,137


In [ ]:
columns_to_fill = ['Temp (°C)', 'Humidity (%)', 'Wind_Speed (km/h)','Soil_Moisture (m³/m³)']

df[columns_to_fill] = df[columns_to_fill].apply(lambda col: col.fillna(col.mean())) 



In [1]:
df.tail()
df.isnull().sum()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings("ignore")

from prophet import Prophet


In [ ]:
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"])
df.drop(["Date", "Time"], axis=1, inplace=True) 

df.set_index("Datetime", inplace=True)

print(df.info())
df.to_csv("Air_Quality_dataset_of_kathmandu_modified.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22536 entries, 2022-08-05 00:00:00 to 2025-02-28 23:00:00
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PM10 (μg/m³)           22536 non-null  float64
 1   PM2.5 (μg/m³)          22536 non-null  float64
 2   CO (μg/m³)             22536 non-null  float64
 3   NO2 (μg/m³)            22536 non-null  float64
 4   SO2 (μg/m³)            22536 non-null  float64
 5   Temp (°C)              22536 non-null  float64
 6   Humidity (%)           22536 non-null  float64
 7   Wind_Speed (km/h)      22536 non-null  float64
 8   Soil_Moisture (m³/m³)  22536 non-null  float64
 9   AQI                    22536 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 1.9 MB
None


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Selecting features and target
features = df.drop(columns=["AQI"])
target = df["AQI"]

# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train a Random Forest Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate model
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("r2_score:", r2_score(y_test, y_pred))

# Plot actual vs predicted AQI
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Actual AQI")
plt.ylabel("Predicted AQI")
plt.title("Actual vs Predicted AQI (Random Forest)")
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Ensure catboost is install
from catboost import CatBoostRegressor

# List of models
models = [
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    BaggingRegressor(),
    ExtraTreesRegressor(),
    XGBRegressor(),
    LGBMRegressor(),
    CatBoostRegressor(verbose=0)
]

# Evaluate each model
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{model.__class__.__name__} - \n MAE: {mean_absolute_error(y_test, y_pred)},\n MSE: {mean_squared_error(y_test, y_pred)},\n r2_score: {r2_score(y_test, y_pred)}")



In [ ]:
import pickle 

# Save the model
with open("model.pkl", "wb") as file:
    pickle.dump(rf, file)


